In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
import os
import collections
from tqdm import tqdm
HERE = %pwd
sys.path.append(os.path.dirname(HERE))

1. Download from https://msnews.github.io/
1. Unzip the file.
1. Place the dataset into `dir_save`

In [2]:
# data directory that save raw data
dir_save = "../data/raw_data/MINDlarge_train"

In [3]:
# item master
df_ = pd.read_csv(f'{dir_save}/news.tsv', delimiter='\t', encoding='utf-8', on_bad_lines='skip', index_col=0)
df_ = df_.reset_index().T.reset_index().T.rename(columns={0 : "itemID"}).set_index("itemID")
df_items = df_[[3, 1, 2, 4]].rename(columns={
    1 : "category",
    2 : "subcategory", 
    3 : "title", 
    4 : "abstract"
})
df_items

,title,category,subcategory,abstract
itemID,,,,
N88753,"The Brands Queen Elizabeth, Prince Charles, an...",lifestyle,lifestyleroyals,"Shop the notebooks, jackets, and more that the..."
N45436,Walmart Slashes Prices on Last-Generation iPads,news,newsscienceandtechnology,Apple's new iPad releases bring big deals on l...
N23144,50 Worst Habits For Belly Fat,health,weightloss,These seemingly harmless habits are holding yo...
N86255,Dispose of unwanted prescription drugs during ...,health,medical,NaN
N93187,The Cost of Trump's Aid Freeze in the Trenches...,news,newsworld,Lt. Ivan Molchanets peeked over a parapet of s...
...,...,...,...,...
N115249,St. Dominic soccer player tries to kick cancer...,sports,more_sports,"Sometimes, what happens on the sidelines can b..."
N64337,Mansion Monday: Contemporary Des Moines home i...,finance,finance-real-estate,Among the perks of this unique Des Moines home...
N100102,How the Sounders won MLS Cup,sports,soccer_epl,"Mark, Jeremiah and Casey were so excited they ..."


In [4]:
# transaction
df_ = pd.read_csv(f'{dir_save}/behaviors.tsv', delimiter='\t', encoding='utf-8', on_bad_lines='skip', index_col=0)
df_ = df_.reset_index().T.reset_index().T
df_records = df_[[1, 3, 4]].rename(columns={
    1 : "userID",
    3 : "log", 
    4 : "candi"})
df_records

,userID,log,candi
index,U87243,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...
0,U598644,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...
1,U532401,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1
2,U593596,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...
3,U239687,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...
...,...,...,...
2232742,U316192,N122359 N37069 N95876 N28787 N73408 N11266 N61321,N113723-0 N123683-1 N5287-0 N76677-0 N53474-0
2232743,U451238,N12575 N93816 N71643 N87236 N87236,N18861-0 N20990-0 N43085-0 N7937-1
2232744,U151246,N27587 N49668,N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...
2232745,U330725,N121944 N91510 N42280 N60061 N63032 N125223 N4...,N18947-0 N88808-1 N10012-0 N38902-0 N33078-0 N...


In [5]:
# select users who have >1 positive (seen) and >9 negative (unseen) news
d_ = dict()
idx = df_records.index.values
for i, t in enumerate(tqdm(df_records["candi"].values)):
    try:
        t = t.split(" ")
        d = {
            "len" : len(t),
            "pos" : len([a for a in t if a.split("-")[1] == "1"])
        }
        d["neg"] = d["len"] - d["pos"]
        d_[idx[i]] = d
    except:
        pass

df = pd.DataFrame(d_).T
df_ = df[(df["pos"] >= 1)*(df["neg"] >= 9)]
df_r = df_records.loc[df_.index]
df_r = df_r.drop_duplicates(subset="userID").set_index("userID")
print(len(df_records["userID"].unique()), len(df_r.index.unique()))

100%|██████████████████████████████████████████████████████| 2232748/2232748 [00:18<00:00, 121396.88it/s]


711222 614234


In [6]:
# directory to save preprocessed data to use experiments
version_prep = "20250403_prep"
dir_save_data = f"../data/preprocessed_data/{version_prep}/MIND"
os.makedirs(dir_save_data, exist_ok=True)

# count news view history
d_ = dict()
users = df_r.index.values
for i, v in enumerate(tqdm(df_r.values)):
    try:
        user = users[i]
        t_log = v[0]
        d_[user] = len(t_log.split(" "))
    except:
        pass
s = pd.Series(d_)

# heavy or light
du = dict()
du["light"] = s[(s>5)*(s<=10)].index.values
du["heavy"] = s[(s>30)*(s<=50)].index.values

di = dict()
for type_user, users in du.items():
    idx = 0
    d_ = dict()
    dr = dict()
    for user in tqdm(users):
        s = df_r.loc[user]
        items_train = s["log"].split(" ")

        # candidate
        items_ = s["candi"].split(" ")
        items_neg = [i.split("-")[0] for i in items_ if "-0" in i]
        items_neg = np.random.choice(items_neg, size=9, replace=False)
        items_pos = [i.split("-")[0] for i in items_ if "-1" in i]
        item_test = np.random.choice(items_pos, size=1, replace=False)[0]
        items_candi = list(items_neg) + [item_test]
        try:
            # whether items are registered in item master
            items = sorted(set(items_train).union(set(items_candi)))
            _ = df_items.loc[items]
            d_[user] = {
                "id_train" : items_train, 
                "id_candi" : items_candi
            }

            # create transaction record
            dr[user] = pd.DataFrame(
                {i : [user, item] for i, item in enumerate(list(items_train) + [item_test])},
                index=["userID", "itemID"]
            ).T
            
            idx += 1
        except:
            pass
    
        if idx == 200 + 5:
            break
    
    # items for experiments
    di.update(d_)
    
    # save
    import pickle
    with open(f"{dir_save_data}/ids_{type_user}.pickle", 'wb') as f:
        pickle.dump(d_, f)
        
    df_r_ = pd.concat(dr.values()).reset_index(drop=True)
    df_r_.to_csv(f"{dir_save_data}/records_{type_user}.csv")  
    
# items that appeared for experiments
items_train = np.unique(np.concatenate([d["id_train"] for d in di.values()]))
items_candi = np.unique(np.concatenate([d["id_candi"] for d in di.values()]))
items = sorted(set(items_train).union(set(items_candi)))

df_items_valid = df_items.loc[items]
df_items_valid.to_csv(f"{dir_save_data}/items_slim.csv")
print("finished")

  0%|▏                                                              | 204/64322 [00:00<01:57, 543.70it/s]


finished


In [7]:
df_r_

,userID,itemID
0,U290933,N14678
1,U290933,N71340
2,U290933,N65259
3,U290933,N92085
4,U290933,N31043
...,...,...
8331,U197579,N82028
8332,U197579,N127629
8333,U197579,N73137
8334,U197579,N108115


In [8]:
df_items_valid

,title,category,subcategory,abstract
itemID,,,,
N100033,Family of slain ISIS hostage says they secretl...,news,newsus,"Kayla Mueller, a humanitarian worker from Ariz..."
N100063,Russian woman convicted by U.S. of being agent...,news,newsworld,Russian woman convicted by U.S. of being agent...
N100073,Why Prince Harry Wore His Remembrance Poppy Di...,lifestyle,lifestyleroyals,Why Prince Harry's Poppy Was Worn on His Cap
N100119,How the Rams' blockbuster trade for Jalen Rams...,sports,football_nfl,Peter King lays out how the Jalen Ramsey block...
N10012,200 Shocking Home Photos You Have to See,lifestyle,lifestylehomeandgarden,"We know a good DIY project when we see it, but..."
...,...,...,...,...
N99846,This stately home is having the ultimate yard ...,finance,finance-real-estate,"Nicky Haslam's English country estate, The Hun..."
N9994,USC Names Mike Bohn New Athletic Director,sports,football_ncaa,"Bohn replaces Lynn Swann, who abruptly resigne..."
N99962,"Report: Patriots trade for Mohamed Sanu, send ...",sports,football_nfl,NaN
